# B-Splines and Exponential Functions

## Exp

The exponential function can be approximated as a weighted sum of the derivatives
of B-splines. More precisely, let an arbitrary order of differentiation be
$q\in{\mathbb{N}}$ and let the degree of the B-spline be large enough to allow for
$q$-times differentiation, by setting it to be $n=p+q$ with $p\in{\mathbb{N}}.$
Then, it holds for $x\in{\mathbb{R}}$ that the B-spline $\beta^{n}$ is such that

$${\mathrm{e}}^{x}=\frac{1}{2^{q+1}}\,\lim_{p\rightarrow\infty}a^{-p-1}\,\sum_{k\in{\mathbb{Z}}}\,\left(1+\sqrt{2}\right)^{k}\,\frac{{\mathrm{d}}^{q}\beta^{p+q}(\left(a\,x-k\right)/2)}{{\mathrm{d}}x^{q}},$$
where we have introduced the constant $a=1/{\mathrm{arcsinh}}(1).$

We verify now this property visually over the range $x\in[-2,2].$ In the top plot,
we show the ground-truth exponential in thin green, and the spline approximation in
thicker blue. In the bottom plot, we show in thicker blue the ratio between the
exponential function and the spline approximation. Although the total degree of the
B-spline grows with the order of differentiation, we observe that the approximation
does not depend on the order of differentiation itself.

In [40]:
# Load the required libraries.
from ipywidgets import interactive
import math
import matplotlib.pyplot as plt
import numpy as np

import splinekit as sk # This library

# Define the plot function
def b_spline_as_exp_plot (
    base_degree = 3,
    diff_order = 0
):
    # Location of the samples
    abscissa = np.linspace(-2, 2, num = 200 + 1, dtype = float)
    # Exponential data
    exp_data = np.exp(abscissa)
    # B-spline approximation of an exponential
    a = 1 / math.asinh(1)
    degree = base_degree + diff_order
    normalization = (a ** (-base_degree - 1)) / (2 ** (diff_order + 1))
    c = 1 + math.sqrt(2)
    spline_data = np.array(
        [
            normalization * math.fsum(
                (c ** k) * sk.diff_b_spline(
                    (a * x - k) / 2,
                    degree = degree,
                    differentiation_order = diff_order
                )
                for k in range(
                    math.floor(a * x) - degree,
                    math.ceil(a * x) + degree + 1
                )
            )
            for x in abscissa
        ],
        dtype = float
    )

    # Spline plot
    plt.subplot(211)
    plt.xlim(-2.025, 2.025)
    plt.ylim(-0.05, 8.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot(abscissa, exp_data, "-C2", linewidth = 0.5)
    plt.plot(abscissa, spline_data, "-C0")

    # Ratio plot
    plt.subplot(212)
    plt.xlim(-2.025, 2.025)
    plt.ylim(0.999, 1.001)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot([-2, 2], [1, 1], "-C2", linewidth = 0.5)
    plt.plot(abscissa, exp_data / spline_data, "-C0")

    # Show the plot
    print(
        "Weighted sum of B-Splines of degree n =", degree,
        "differentiated", diff_order, "times"
    )
    plt.show()

# Interact with the degree
interactive(b_spline_as_exp_plot, base_degree = (0, 9), diff_order = (0, 5))

interactive(children=(IntSlider(value=3, description='base_degree', max=9), IntSlider(value=0, description='di…

## Erf

Polynomial B-splines are bump-like functions that converge to a Gaussian
when the degree grows. More precisely, $\forall x\in{\mathbb{R}}$ it holds that

$$\frac{1}{\sigma_{0}\,\sqrt{2\,\uppi}}\,{\mathrm{e}}^{-\frac{x^{2}}{2\,\sigma_{0}^{2}}}=\lim_{n\rightarrow\infty}\beta^{n}(x\,\sqrt{n+1})\,\sqrt{n+1},$$
where we have indroduced the constant $\sigma_{0}=1/\sqrt{12}.$  Therefore, it is
without surprise that the integral of a spline converges to the ${\mathrm{Erf}}$
function as the degree grows, too. Indeed, one has $\forall x\in{\mathbb{R}}$ that

$${\mathrm{Erf}}(x)=-1+2\,\lim_{n\rightarrow\infty}\int_{0}^{x\,\sqrt{\frac{n+1}{6}}}\,
\beta^{n}(y)\,{\mathrm{d}}y.$$ We verify now this property visually over the range $x\in[-3,3].$ In the top plot, we show the ground-truth ${\mathrm{Erf}}$ function in
thin green, and the spline approximation in thicker blue. In the bottom plot, we
show in thicker blue the difference between the ${\mathrm{Erf}}$ function and the
spline approximation.

In [29]:
# Load the required libraries.
from ipywidgets import interactive
import math
import matplotlib.pyplot as plt
import numpy as np

import splinekit as sk # This library

# Define the plot function
def b_spline_as_erf_plot (
    degree = 3
):
    # Location of the samples
    abscissa = np.linspace(-3, 3, num = 200 + 1, dtype = float)
    # Erf data
    erf_data = np.array([math.erf(x) for x in abscissa], dtype = float)
    # B-spline approximation of an Erf
    spline_data = np.array(
        [
            -1 + 2 * sk.integrated_b_spline(x * math.sqrt((degree + 1) / 6), degree)
            for x in abscissa
        ],
        dtype = float
    )

    # Spline plot
    plt.subplot(211)
    plt.xlim(-3.025, 3.025)
    plt.ylim(-1.05, 1.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot(abscissa, erf_data, "-C2", linewidth = 0.5)
    plt.plot(abscissa, spline_data, "-C0")

    # Difference plot
    plt.subplot(212)
    plt.xlim(-3.025, 3.025)
    plt.ylim(-0.05, 0.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot([-3, 3], [0, 0], "-C2", linewidth = 0.5)
    plt.plot(abscissa, erf_data - spline_data, "-C0")

    # Show the plot
    plt.show()

# Interact with the degree
interactive(b_spline_as_erf_plot, degree = (0, 9))

interactive(children=(IntSlider(value=3, description='degree', max=9), Output()), _dom_classes=('widget-intera…

*   As exercise, we propose to modify the code of the ${\mathrm{Erf}}$
    case to verify visually that a the B-spline of degree $n\in{\mathbb{N}}$
    converges to a Gaussian when the degree grows. As icing on the cake, we
    suggest to perform the visualization over the support of the approximating
    B-spline.

We give below the solution to this exercise.

In [ ]:
# Spline approximation to a Gaussian

# Load the required libraries.
from ipywidgets import interactive
import math
import matplotlib.pyplot as plt
import numpy as np

import splinekit as sk # This library

# Define the plot function
def b_spline_as_gaussian_plot (
    degree = 3
):
    # Support of the approximating B-spline
    supp = sk.b_spline_support(degree)
    min_x = supp.infimum / math.sqrt(degree + 1)
    max_x = supp.supremum / math.sqrt(degree + 1)
    # Location of the samples
    abscissa = np.linspace(min_x, max_x, num = 200 + 1, dtype = float)
    # Gaussian data
    gaussian_data = np.array(
        [
            (1 / math.sqrt((1 / 12) * 2 * math.pi)) * math.exp(-(x ** 2) / (2 / 12))
            for x in abscissa
        ],
        dtype = float
    )
    # B-spline approximation of a Gaussian
    spline_data = np.array(
        [
            sk.b_spline(x * math.sqrt(degree + 1), degree) * math.sqrt(degree + 1)
            for x in abscissa
        ],
        dtype = float
    )

    # Spline plot
    plt.subplot(211)
    plt.xlim(min_x - 0.025, max_x + 0.025)
    plt.ylim(-0.05, 1 / math.sqrt((1 / 12) * 2 * math.pi) + 0.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot(abscissa, gaussian_data, "-C2", linewidth = 0.5)
    plt.plot(abscissa, spline_data, "-C0")

    # Ratio plot
    plt.subplot(212)
    plt.xlim(min_x - 0.025, max_x + 0.025)
    plt.ylim(-0.05, 0.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot([min_x, max_x], [1, 1], "-C2", linewidth = 0.5)
    plt.plot(abscissa, gaussian_data - spline_data, "-C0")

    # Show the plot
    plt.show()

# Interact with the degree
interactive(b_spline_as_gaussian_plot, degree = (0, 9))